In [ ]:
#1.Write a program to implement Needleman-Wunsch for proteins
import numpy as np
blosum50 = np.loadtxt('blosum50.txt')
string1 = 'HEAGAWGHEE'
string2 = 'PAWHEAE'
#string1 = 'PQPTTPVSSFTSGSMLGRTDTALTNTYSAL'
#string2 = 'PSPTMEAVEASTASHPHSTSSYFATTYYHL' 
idx = 'ARNDCQEGHILKMFPSTWYV'
d = -8
cols = len(string1) + 1
rows = len(string2) + 1
F = np.array(np.zeros((rows,cols)))
for i in range(0,cols):
    F[0,i] = d*i
for j in range(0,rows):
    F[j,0] = d*j
for i in range(0,rows-1):
    for j in range(0,cols-1):
        match = F[i,j] + blosum50[idx.index(string2[i]),idx.index(string1[j])]
        delete = F[i,j+1] + d
        insert = F[i+1,j] + d
        F[i+1,j+1] = max(match,delete,insert)
print(F)
AlignmentA = []
AlignmentB = []
while(rows > 1 or cols > 1):
    if(rows > 1 and cols > 1 and F[rows-1,cols-1] == F[rows-2,cols-2] + blosum50[idx.index(string2[rows-2]),idx.index(string1[cols-2])]):
        AlignmentA.append(string2[rows-2])
        AlignmentB.append(string1[cols-2])
        rows = rows - 1
        cols = cols - 1
    elif(rows > 1 and F[rows-1,cols-1] == F[rows-2,cols-1] + d):
        AlignmentA.append(string2[rows-2])
        AlignmentB.append('-')
        rows = rows - 1
    else:
        AlignmentA.append('-')
        AlignmentB.append(string1[cols-2])
        cols = cols - 1
AlignmentA = ''.join(reversed(AlignmentA))
AlignmentB = ''.join(reversed(AlignmentB))
print(AlignmentB + '\n' + AlignmentA)

In [ ]:
#2.Modify your program to implement the Smith-Waterman algorithm
import numpy as np
blosum50 = np.loadtxt('blosum50.txt')
string1 = 'HEAGAWGHEE'
string2 = 'PAWHEAE'
#string1 = 'MQNSHSGVNQLGGVFVNGRPLPDSTRQKIVELAHSGARPCDISRILQVSNGCVSKILGRY'
#string2 = 'TDDECHSGVNQLGGVFVGGRPLPDSTRQKIVELAHSGARPCDISRI'
idx = 'ARNDCQEGHILKMFPSTWYV'
d = -8
cols = len(string1) + 1
rows = len(string2) + 1
posX = 0
posY = 0
F = np.array(np.zeros((rows,cols)))
for i in range(0,rows-1):
    for j in range(0,cols-1):
        match = F[i,j] + blosum50[idx.index(string2[i]),idx.index(string1[j])]
        delete = F[i,j+1] + d
        insert = F[i+1,j] + d
        F[i+1,j+1] = max(match,delete,insert,0)
print(F)
for p in range(0,rows):
    for k in range(0,cols):
        if(F[p,k] == np.amax(F)):
            posX = p
            posY = k
print('Indices of maximum value:',posX,posY)
AlignmentA = []
AlignmentB = []
while(posX > 0 or posY > 0):
    if(posX > 0 and cols > 0 and F[posX,posY] == F[posX-1,posY-1] + blosum50[idx.index(string2[posX-1]),idx.index(string1[posY-1])]):
        AlignmentA.append(string2[posX-1])
        AlignmentB.append(string1[posY-1])
        posX = posX - 1
        posY = posY - 1
        print(posX,posY)
        if(F[posX,posY]==0):
            break        
    elif(posX > 0 and F[posX,posY] == F[posX-1,posY] + d):
        AlignmentA.append(string2[posX-1])
        AlignmentB.append('-')
        posX = posX - 1
        print(posX,posY)
    else:
        AlignmentA.append('-')
        AlignmentB.append(string1[posY-1])
        posY = posY - 1
        print(posX,posY)
AlignmentA = ''.join(reversed(AlignmentA))
AlignmentB = ''.join(reversed(AlignmentB))
print(AlignmentB + '\n' + AlignmentA)

In [19]:
#3.viterbi algorithm
import math as m
import numpy as np

# probability of state H
HH = round(m.log(2,0.9998),2);
#HH = -1;
# probability of state D
DD = round(m.log(2,0.9997),2);
#DD = -0.737;
# probability of transmitting state
HD = round(m.log(2,0.0002),2);
DH = round(m.log(2,0.0003),2);
'''
HD = -1;
DH = -1.322;
'''
# define initial emission probability and store them in a table
idx = "ATCG";
idxTable = np.matrix(np.zeros((2,4)));
idxTable[0,0] = round(m.log(2,0.2698),2); # HA
idxTable[0,1] = round(m.log(2,0.3237),2); # HT
idxTable[0,2] = round(m.log(2,0.2080),2); # HC
idxTable[0,3] = round(m.log(2,0.1985),2); # HG
idxTable[1,0] = round(m.log(2,0.2459),2); # DA
idxTable[1,1] = round(m.log(2,0.2079),2); # DT
idxTable[1,2] = round(m.log(2,0.2478),2); # DC
idxTable[1,3] = round(m.log(2,0.2984),2); # DG
'''
idxTable[0,0] = -2.322;
idxTable[0,1] = -2.322;
idxTable[0,2] = -1.737;
idxTable[0,3] = -1.737;
idxTable[1,0] = -1.737;
idxTable[1,1] = -1.737;
idxTable[1,2] = -2.322;
idxTable[1,3] = -2.322;
'''

# create probability table
sequence = "GGGCGGCGACCTCGCGGGTTTTCGCTATTTATGAAAATTTTCCGGTTTAAGGCGTTTCCGTTCTTCTTCG"
#sequence = "GGCACTGAA"
cols = len(sequence);
rows = 2;
proTable = np.array(np.zeros((rows,cols)));

# calculate probability -------------------------------
# calculate the first probability of each state
# each state has 0.5 probability to start, log(2,0.5) = -1
proTable[0,0] = -1 + idxTable[0,idx.index(sequence[0])];
proTable[1,0] = -1 + idxTable[1,idx.index(sequence[0])];
# calculate the rest probabilities
for i in range(1,cols):    
    j = 0;   
    proTable[j,i] = idxTable[j,idx.index(sequence[i])] + max(proTable[j,i-1]+HH, proTable[j+1,i-1]+DH); # state H
    proTable[j+1,i] = idxTable[j+1,idx.index(sequence[i])] + max(proTable[j,i-1]+HD, proTable[j+1,i-1]+DD); # state D
print('Probability table is: ');
print(proTable)
print('\n');

# back-tracking -----------------------
alignment = [];
k = cols;
# probability of last position 
probMax = max(proTable[0,cols-1],proTable[1,cols-1]);
if (proTable[0,cols-1] == probMax):
    alignment.append('H');
else:
    alignment.append('D');
# back-tracking
while (k > 1):  
    k = k - 1;
    probMax = max(proTable[0,k],proTable[1,k]);
    if (probMax == idxTable[0,idx.index(sequence[k])] + max(proTable[0,k-1]+HH, proTable[1,k-1]+DH)):
        if (proTable[0,k-1]+HH == max(proTable[0,k-1]+HH, proTable[1,k-1]+DH)):
            alignment.append('H');
        else:
            alignment.append('D');
    else:
        if (proTable[0,k-1]+HD == max(proTable[0,k-1]+HD, proTable[1,k-1]+DD)):
            alignment.append('H');
        else:
            alignment.append('D');
alignment = ''.join(reversed(alignment));
print('The most likely path is:'+ '\n' + alignment);

Probability table is: 
[[ -1.43  -2.09  -2.6   -3.27  -3.7   -4.44  -4.88  -5.54  -6.15  -6.64
   -7.26  -7.92  -8.31  -9.02  -9.49 -10.12 -10.66 -11.29 -12.01 -12.51
  -13.23 -13.73 -14.28 -14.83 -15.46 -16.11 -16.6  -17.38 -17.82 -18.6
  -18.96 -19.87 -20.   -21.14 -21.19 -22.33 -22.38 -23.6  -23.6  -24.82
  -24.82 -25.87 -25.93 -26.97 -27.1  -28.32 -28.32 -29.54 -29.46 -30.73
  -30.55 -31.9  -31.73 -33.   -33.08 -34.22 -34.3  -35.27 -35.41 -36.37
  -36.76 -37.59 -37.81 -38.87 -39.03 -39.92 -40.31 -41.14 -41.36 -42.24]
 [ -1.57  -2.08  -2.74  -3.18  -3.92  -4.35  -5.02  -5.53  -6.11  -6.73
   -7.22  -7.78  -8.5   -8.96  -9.6  -10.14 -10.77 -11.31 -11.81 -12.53
  -13.03 -13.75 -14.31 -14.93 -15.41 -15.98 -16.68 -17.12 -17.9  -18.34
  -19.17 -19.48 -20.52 -20.57 -21.71 -21.76 -22.9  -22.9  -24.12 -24.12
  -25.34 -25.4  -26.45 -26.58 -27.62 -27.62 -28.84 -28.84 -30.11 -30.03
  -31.38 -31.2  -32.48 -32.38 -33.52 -33.6  -34.74 -34.88 -35.85 -36.06
  -36.89 -37.28 -38.17 -38.33 -39.39 -39.